In [6]:
import os
import torch
from torch import nn, optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the full dataset
data_dir = 'chessDataset'
full_dataset = datasets.ImageFolder(data_dir, transform=transform)

# Define class names based on folder structure
class_names = [
    'black_bishop', 'black_king', 'black_knight', 'black_pawn', 'black_queen', 'black_rook',  # Black pieces
    'empty',  # Empty squares
    'white_bishop', 'white_king', 'white_knight', 'white_pawn', 'white_queen', 'white_rook'  # White pieces
]

# Use the entire dataset for training
train_data = full_dataset
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

# Define model (using a pre-trained model like ResNet for feature extraction)
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(class_names))  # Adjust output layer to number of classes
model = model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Training loop
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        accuracy = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
            # Calculate accuracy
            _, preds = torch.max(outputs, 1)
            accuracy += (preds == labels).sum().item()
        
        # Print epoch results
        print(f"Epoch {epoch+1}/{epochs}")
        print(f"Train Loss: {running_loss/len(train_loader):.4f}")
        print(f"Train Accuracy: {accuracy / len(train_data):.4f}")

# Train the model
train_model(model, train_loader, criterion, optimizer, epochs=10)

# Save the model
torch.save(model.state_dict(), 'chess_piece_classifier.pth')


Epoch 1/10
Train Loss: 0.7088
Train Accuracy: 0.8302
Epoch 2/10
Train Loss: 0.1060
Train Accuracy: 0.9950
Epoch 3/10
Train Loss: 0.0444
Train Accuracy: 0.9994
Epoch 4/10
Train Loss: 0.0272
Train Accuracy: 0.9994
Epoch 5/10
Train Loss: 0.0230
Train Accuracy: 0.9994
Epoch 6/10
Train Loss: 0.0156
Train Accuracy: 0.9994
Epoch 7/10
Train Loss: 0.0137
Train Accuracy: 0.9994
Epoch 8/10
Train Loss: 0.0129
Train Accuracy: 0.9994
Epoch 9/10
Train Loss: 0.0164
Train Accuracy: 0.9994
Epoch 10/10
Train Loss: 0.0124
Train Accuracy: 0.9994
